In [2]:
!pip install redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 KB 2.3 MB/s eta 0:00:00a 0:00:01


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import redis

In [ ]:
# Конфигурация Valkey
valkey_config = {
    "host": "valkey",
    "port": 6379,
    "db": 0,
    "password": "bigdata2024",  # пароль из docker-compose
    "decode_responses": True
}

spark = SparkSession.builder \
    .appName("ValkeyETL") \
    .getOrCreate()

In [5]:
def test_valkey_connection():
    """Тестирование подключения к Valkey"""
    try:
        r = redis.Redis(**valkey_config)
        r.ping()
        print("✓ Valkey подключение успешно")
        return True
    except Exception as e:
        print(f"✗ Ошибка подключения к Valkey: {e}")
        return False

test_valkey_connection()

✓ Valkey подключение успешно


True

In [6]:
pg_url = "jdbc:postgresql://postgres:5432/bober_db"
pg_properties = {"user": "bober", "password": "bober", "driver": "org.postgresql.Driver"}
df = spark.read.jdbc(url=pg_url, table="mock_data", properties=pg_properties)

# Загружаем все таблицы звезды
fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
dim_product = spark.read.jdbc(url=pg_url, table="dim_product", properties=pg_properties)
dim_customer = spark.read.jdbc(url=pg_url, table="dim_customer", properties=pg_properties)
dim_store = spark.read.jdbc(url=pg_url, table="dim_store", properties=pg_properties)
dim_supplier = spark.read.jdbc(url=pg_url, table="dim_supplier", properties=pg_properties)
dim_date = spark.read.jdbc(url=pg_url, table="dim_date", properties=pg_properties)
dim_date.head(1)

[Row(full_date=datetime.date(2021, 1, 1), date_id=1, year=2021, month=1, day=1, quarter=1)]

In [7]:
def save_dataframe_to_valkey(df, key_name, key_column="id"):
    """
    Сохраняет DataFrame в Valkey как Hash
    """
    df.write \
        .format("org.apache.spark.sql.redis") \
        .option("table", key_name) \
        .option("key.column", key_column) \
        .mode("overwrite") \
        .save()

def save_key_value_to_valkey(df, key_prefix, key_column, value_column):
    """
    Сохраняет пары ключ-значение в Valkey
    """
    df.select(
        concat(lit(key_prefix), col(key_column)).alias("key"),
        col(value_column).alias("value")
    ).write \
        .format("org.apache.spark.sql.redis") \
        .option("table", "temp") \
        .option("key.column", "key") \
        .mode("append") \
        .save()

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

# Конфигурация Valkey
VALKEY_CONFIG = {
    "host": "valkey",
    "port": 6379,
    "db": 0,
    "password": "bigdata2024",
    "decode_responses": True
}

def test_valkey_connection():
    """Тестирование подключения к Valkey"""
    try:
        import redis
        r = redis.Redis(**VALKEY_CONFIG)
        r.ping()
        print("✓ Valkey подключение успешно")
        return True
    except Exception as e:
        print(f"✗ Ошибка подключения к Valkey: {e}")
        return False

def save_dataframe_to_valkey(df, key_prefix, id_column="id"):
    """
    Сохраняет DataFrame в Valkey используя RDD
    """
    # Создаем broadcast переменную с конфигурацией
    config_broadcast = spark.sparkContext.broadcast(VALKEY_CONFIG)
    
    def save_partition(partition):
        import redis
        
        # Получаем конфигурацию из broadcast переменной
        config = config_broadcast.value
        
        # Создаем подключение для каждой партиции
        r = redis.Redis(**config)
        
        saved_count = 0
        for row in partition:
            try:
                # Создаем ключ
                key = f"{key_prefix}:{row[id_column]}"
                
                # Сохраняем все поля как hash
                row_dict = row.asDict()
                for field, value in row_dict.items():
                    if value is not None:
                        r.hset(key, field, str(value))
                
                saved_count += 1
                
            except Exception as e:
                print(f"Ошибка при сохранении {key}: {e}")
        
        r.close()
        print(f"Сохранено записей в партиции: {saved_count}")
    
    # Сохраняем через RDD
    df.rdd.foreachPartition(save_partition)
    print(f"✅ Успешно сохранено в Valkey с префиксом: {key_prefix}")

def create_product_sales_dashboard(fact, dim_product):
    """
    Витрина 1: Продажи по продуктам
    """
    print("=== Создание витрины продаж по продуктам ===")
    
    # Топ-10 самых продаваемых продуктов
    top_products = fact.join(dim_product, "product_id") \
        .groupBy("product_id", "name", "category") \
        .agg(
            sum("sale_quantity").alias("total_quantity"),
            sum("sale_total_price").alias("total_revenue"),
            avg("rating").alias("avg_rating"),
            sum("reviews").alias("total_reviews")
        ) \
        .orderBy(desc("total_quantity")) \
        .limit(10)
    
    print("Топ-10 продуктов:")
    top_products.show()
    
    # Сохраняем в Valkey
    save_dataframe_to_valkey(top_products, "top_products", "product_id")
    
    # Общая выручка по категориям
    category_revenue = fact.join(dim_product, "product_id") \
        .groupBy("category") \
        .agg(
            sum("sale_total_price").alias("category_revenue"),
            count("sale_id").alias("sales_count")
        ) \
        .orderBy(desc("category_revenue"))
    
    print("Выручка по категориям:")
    category_revenue.show()
    
    # Сохраняем категории (создаем искусственный ID)
    category_revenue_with_id = category_revenue.withColumn(
        "category_id", 
        md5(col("category"))
    )
    save_dataframe_to_valkey(category_revenue_with_id, "category_revenue", "category_id")
    
    print("✅ Витрина продаж по продуктам успешно создана!")

def create_customer_sales_dashboard(fact, dim_customer):
    """
    Витрина 2: Продажи по клиентам
    """
    print("=== Создание витрины продаж по клиентам ===")
    
    # Топ-10 клиентов
    top_customers = fact.join(dim_customer, "customer_id") \
        .groupBy("customer_id", "first_name", "last_name", "country") \
        .agg(
            sum("sale_total_price").alias("total_spent"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_spent")) \
        .limit(10)
    
    print("Топ-10 клиентов:")
    top_customers.show()
    
    save_dataframe_to_valkey(top_customers, "top_customers", "customer_id")
    
    # Распределение по странам
    country_distribution = fact.join(dim_customer, "customer_id") \
        .groupBy("country") \
        .agg(count("customer_id").alias("customer_count")) \
        .orderBy(desc("customer_count"))
    
    country_distribution_with_id = country_distribution.withColumn(
        "country_id", 
        md5(col("country"))
    )
    save_dataframe_to_valkey(country_distribution_with_id, "customers_by_country", "country_id")
    
    print("✅ Витрина продаж по клиентам успешно создана!")

def create_time_sales_dashboard(fact, dim_date):
    """
    Витрина 3: Продажи по времени
    """
    print("=== Создание витрины продаж по времени ===")
    
    # Месячные тренды
    monthly_trends = fact.join(dim_date, "date_id") \
        .groupBy("year", "month") \
        .agg(
            sum("sale_total_price").alias("monthly_revenue"),
            count("sale_id").alias("monthly_orders"),
            avg("sale_total_price").alias("avg_order_size")
        ) \
        .orderBy("year", "month")
    
    monthly_trends = monthly_trends.withColumn(
        "period_id", 
        concat(col("year"), lit("-"), lpad(col("month"), 2, "0"))
    )
    
    print("Месячные тренды:")
    monthly_trends.show()
    
    save_dataframe_to_valkey(monthly_trends, "monthly_sales", "period_id")
    print("✅ Витрина продаж по времени успешно создана!")

def create_store_sales_dashboard(fact, dim_store):
    """
    Витрина 4: Продажи по магазинам
    """
    print("=== Создание витрины продаж по магазинам ===")
    
    # Топ-5 магазинов
    top_stores = fact.join(dim_store, "store_id") \
        .groupBy("store_id", "store_name", "store_city", "store_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 магазинов:")
    top_stores.show()
    
    save_dataframe_to_valkey(top_stores, "top_stores", "store_id")
    print("✅ Витрина продаж по магазинам успешно создана!")

def create_supplier_sales_dashboard(fact, dim_supplier, dim_product):
    """
    Витрина 5: Продажи по поставщикам
    """
    print("=== Создание витрины продаж по поставщикам ===")
    
    # Топ-5 поставщиков
    top_suppliers = fact.join(dim_supplier, "supplier_id") \
        .join(dim_product, "product_id") \
        .groupBy("supplier_id", "supplier_name", "supplier_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            avg("price").alias("avg_product_price")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 поставщиков:")
    top_suppliers.show()
    
    save_dataframe_to_valkey(top_suppliers, "top_suppliers", "supplier_id")
    print("✅ Витрина продаж по поставщикам успешно создана!")

def create_product_quality_dashboard(dim_product, fact):
    """
    Витрина 6: Качество продукции
    """
    print("=== Создание витрины качества продукции ===")
    
    # Продукты с продажами и рейтингами
    product_ratings = dim_product \
        .join(fact.groupBy("product_id").agg(sum("sale_quantity").alias("total_sold")), "product_id", "left") \
        .select(
            "product_id", "name", "category", "rating", "reviews",
            coalesce("total_sold", lit(0)).alias("total_sold")
        ) \
        .filter(col("rating").isNotNull()) \
        .orderBy(desc("rating"))
    
    # Топ-10 по рейтингу
    top_rated = product_ratings.limit(10)
    print("Топ-10 продуктов по рейтингу:")
    top_rated.show()
    
    # Худшие по рейтингу
    worst_rated = product_ratings.orderBy("rating").limit(10)
    print("Худшие 10 продуктов по рейтингу:")
    worst_rated.show()
    
    save_dataframe_to_valkey(top_rated, "top_rated_products", "product_id")
    save_dataframe_to_valkey(worst_rated, "worst_rated_products", "product_id")
    
    print("✅ Витрина качества продукции успешно создана!")

def run_complete_valkey_etl():
    """Запуск полного ETL процесса в Valkey"""
    
    global spark
    
    # Создаем Spark сессию
    spark = SparkSession.builder \
        .appName("ValkeyETL") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    # Тестируем подключение
    if not test_valkey_connection():
        print("❌ Не удалось подключиться к Valkey. Проверьте конфигурацию.")
        spark.stop()
        return
    
    print("🚀 Начало ETL процесса в Valkey...")
    
    # Конфигурация PostgreSQL
    pg_url = "jdbc:postgresql://postgres:5432/bober_db"
    pg_properties = {
        "user": "bober",
        "password": "bober", 
        "driver": "org.postgresql.Driver"
    }
    
    try:
        # Загружаем данные из PostgreSQL
        print("📥 Загрузка данных из PostgreSQL...")
        
        fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
        dim_product = spark.read.jdbc(url=pg_url, table="dim_product", properties=pg_properties)
        dim_customer = spark.read.jdbc(url=pg_url, table="dim_customer", properties=pg_properties)
        dim_store = spark.read.jdbc(url=pg_url, table="dim_store", properties=pg_properties)
        dim_supplier = spark.read.jdbc(url=pg_url, table="dim_supplier", properties=pg_properties)
        dim_date = spark.read.jdbc(url=pg_url, table="dim_date", properties=pg_properties)
        
        print("✅ Данные успешно загружены из PostgreSQL")
        
        # Показываем схемы данных для отладки
        print("Схема fact_sales:")
        fact.printSchema()
        
        print("Схема dim_product:")
        dim_product.printSchema()
        
        # Создаем все витрины
        create_product_sales_dashboard(fact, dim_product)
        create_customer_sales_dashboard(fact, dim_customer)
        create_time_sales_dashboard(fact, dim_date)
        create_store_sales_dashboard(fact, dim_store)
        create_supplier_sales_dashboard(fact, dim_supplier, dim_product)
        create_product_quality_dashboard(dim_product, fact)
        
        print("🎉 Все витрины успешно созданы в Valkey!")
        
    except Exception as e:
        print(f"❌ Ошибка в ETL процессе: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        spark.stop()
        print("🛑 Spark сессия остановлена")

# Запуск
if __name__ == "__main__":
    run_complete_valkey_etl()

✓ Valkey подключение успешно
🚀 Начало ETL процесса в Valkey...
📥 Загрузка данных из PostgreSQL...
✅ Данные успешно загружены из PostgreSQL
Схема fact_sales:
root
 |-- sale_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- pet_id: integer (nullable = true)
 |-- seller_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- supplier_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- sale_quantity: integer (nullable = true)
 |-- sale_total_price: decimal(12,2) (nullable = true)

Схема dim_product:
root
 |-- product_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: decimal(10,2) (nullable = true)
 |-- weight: decimal(8,2) (nullable = true)
 |-- color: string (nullable = true)
 |-- size: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- material: string (nullable = true)
 |-- description: stri

Сохранено записей в партиции: 10                                    (0 + 1) / 1]
                                                                                

✅ Успешно сохранено в Valkey с префиксом: top_products
Выручка по категориям:
+--------+----------------+-----------+
|category|category_revenue|sales_count|
+--------+----------------+-----------+
|     Toy|      8643140.80|      34080|
|    Cage|      8407817.76|      33278|
|    Food|      8259782.24|      32682|
+--------+----------------+-----------+



Сохранено записей в партиции: 3


✅ Успешно сохранено в Valkey с префиксом: category_revenue
✅ Витрина продаж по продуктам успешно создана!
=== Создание витрины продаж по клиентам ===
Топ-10 клиентов:
+-----------+----------+---------+-------+-----------+------------+---------------+
|customer_id|first_name|last_name|country|total_spent|total_orders|avg_order_value|
+-----------+----------+---------+-------+-----------+------------+---------------+
|        611|     Allin|   O'Hare| Sweden|    4005.98|          10|     400.598000|
|        611|  Claudian|   Gleave|  China|    4005.98|          10|     400.598000|
|        611|   Martica|    Dooly| Poland|    4005.98|          10|     400.598000|
|        611|   Randolf|  Rillatt| Brazil|    4005.98|          10|     400.598000|
|        611|   Devonna|  Biswell| Russia|    4005.98|          10|     400.598000|
|        611|    Hannie|  Braddon|  China|    4005.98|          10|     400.598000|
|        611|      Ford|   Freake|  Kenya|    4005.98|          10|     400.5

...

✅ Успешно сохранено в Valkey с префиксом: top_stores
✅ Витрина продаж по магазинам успешно создана!
=== Создание витрины продаж по поставщикам ===
Топ-5 поставщиков:
+-----------+-------------+--------------------+-------------+-----------------+
|supplier_id|supplier_name|    supplier_country|total_revenue|avg_product_price|
+-----------+-------------+--------------------+-------------+-----------------+
|       4746|       Meezzy|           Indonesia|      8413.40|        34.599000|
|       6447|       Rhynyx|Republic of the C...|      6390.40|        62.372000|
|       7899|    Topicblab|               China|      5109.00|        34.599000|
|        726|   Brainverse|             Ireland|      4998.50|        47.134000|
|       3438|        Jamia|              Russia|      4998.00|        56.388000|
+-----------+-------------+--------------------+-------------+-----------------+



Сохранено записей в партиции: 5


✅ Успешно сохранено в Valkey с префиксом: top_suppliers
✅ Витрина продаж по поставщикам успешно создана!
=== Создание витрины качества продукции ===
Топ-10 продуктов по рейтингу:
+----------+---------+--------+------+-------+----------+
|product_id|     name|category|rating|reviews|total_sold|
+----------+---------+--------+------+-------+----------+
|       796|Bird Cage|     Toy|   5.0|     12|        41|
|       602| Dog Food|    Food|   5.0|    991|        51|
|       530| Dog Food|    Cage|   5.0|    352|        58|
|       392| Dog Food|     Toy|   5.0|    567|        78|
|       939|  Cat Toy|    Food|   5.0|    536|        72|
|        65|  Cat Toy|     Toy|   5.0|    838|        56|
|        28| Dog Food|     Toy|   5.0|    221|        43|
|       321|Bird Cage|    Food|   5.0|    716|        43|
|       210| Dog Food|    Food|   5.0|    107|        46|
|       914|  Cat Toy|    Cage|   5.0|    705|        43|
+----------+---------+--------+------+-------+----------+

Худшие 1

Сохранено записей в партиции: 10
Сохранено записей в партиции: 10


✅ Успешно сохранено в Valkey с префиксом: worst_rated_products
✅ Витрина качества продукции успешно создана!
🎉 Все витрины успешно созданы в Valkey!
🛑 Spark сессия остановлена
